First, packages are imported.

In [1]:
#   import packages
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import pickle
import functools

from ema_workbench import (
    Model,
    Policy,
    Scenario,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
    util,
    ScalarOutcome,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.em_framework.optimization import ArchiveLogger, EpsilonProgress
from ema_workbench.em_framework import parameters_from_csv
from ema_workbench.em_framework.optimization import EpsilonProgress
from ema_workbench.em_framework.evaluators import BaseEvaluator
from ema_workbench.analysis import parcoords

### Choice of robustness metrics:


| Robustness metric 	 |    kind 	     |
|:-------------------:|:-------------:|
|   Damage Cost          	   | Minimize    	 |
|          	Deaths Score          |       	Minimize       |
|          	 Dike Invest Cost         |       	Minimize       |
|          RfR Invest Cost       |       	Minimize       |
|          	Evacuation Cost        |       	Minimize       |


Then, the model is loaded, and logging is set.

In [2]:
dike_model, planning_steps = get_model_for_problem_formulation(5)
ema_logging.log_to_stderr(ema_logging.INFO)

<Logger EMA (DEBUG)>

The results from the basecase open exploration, which includes the worst scenarios, are loaded into a Pandas DataFrame.

In [3]:
open_exploration_results = pd.read_pickle(r'generated_datasets\open_exploration_base_policy_worst_scenarios.pkl')

To calculate the robustness metrics, which are defined as the total annual damage, total expected number of deaths, total dike investment costs, total room for the river costs and total expected evacuation costs, lists of these variables are saved for later use.

In [4]:
# List the names of vars to make `robustness_functions` a bit more read-able
var_list_damage = ['A.1_Expected Annual Damage',
                   'A.2_Expected Annual Damage',
                   'A.3_Expected Annual Damage',
                   'A.4_Expected Annual Damage',
                   'A.5_Expected Annual Damage',]
var_list_deaths = ['A.1_Expected Number of Deaths',
                   'A.2_Expected Number of Deaths',
                   'A.3_Expected Number of Deaths',
                   'A.4_Expected Number of Deaths',
                   'A.5_Expected Number of Deaths',]
var_list_dike = ['A.1_Dike Investment Costs',
                 'A.2_Dike Investment Costs',
                 'A.3_Dike Investment Costs',
                 'A.4_Dike Investment Costs',
                 'A.5_Dike Investment Costs',]
var_list_rfr = ['RfR Total Costs',]
var_list_evac = ['Expected Evacuation Costs',]

Then, the robustness metric is initiated into a function. For this, we chose LEG UIT.
Furthermore, maximalization and minimalization functions are initiated to use in the optimization process.

In [5]:
def robust_sum(*data):
    sumsumsum = sum(sum(sum(data)))

    # we moeten nog een bron vinden waarom we deze metric gebruiken!
    mean = np.mean(sumsumsum)
    iqr = sp.stats.iqr(sumsumsum) + mean * 0.005
    result = mean + iqr

    return result

Then, we make the robustness functions and add constraints. We chose minimize and maximize, most conservative (Kwakkel, J. H., Eker, S., & Pruyt, E. (2016). How robust is a robust policy? Comparing alternative robustness metrics for robust decision-making. Robustness analysis in decision aiding, optimization, and analytics, 221-237.)

In [6]:
# robustness metrics
maximize = ScalarOutcome.MAXIMIZE
minimize = ScalarOutcome.MINIMIZE

robustness_functions = [
    ScalarOutcome('Damage Cost',
                  variable_name=var_list_damage,
                  function=robust_sum,
                  kind=minimize,
                  ),
    ScalarOutcome('Deaths Score',
                  variable_name=var_list_deaths,
                  function=robust_sum,
                  kind=minimize,
                  ),
    ScalarOutcome('Dike Invest Cost',
                  variable_name=var_list_dike,
                  function=robust_sum,
                  kind=minimize,
                  ),
    ScalarOutcome("RfR Invest Cost",
                  kind=minimize,
                  function=robust_sum,
                  variable_name="RfR Total Costs"),
    ScalarOutcome("Evacuation Cost",
                  kind=minimize,
                  function=robust_sum,
                  variable_name="Expected Evacuation Costs"),
]

These scenarios are then loaded into a model variable.

In [7]:
scenarios = []
for row_number in range(open_exploration_results.shape[0]):
    scenarios.append(
        Scenario(name=row_number, **open_exploration_results.iloc[row_number, :16].to_dict())
    )

Convergence metrics are loaded

In [8]:
convergence_metrics = [
    ArchiveLogger(
        "./archives",
        [l.name for l in dike_model.levers],
        [o.name for o in robustness_functions],
        base_filename="robust_optimization_test.tar.gz",
    ),
    EpsilonProgress(),
]

Finally, the model variables are set.

In [9]:
# general input
nfe = int(25) #make this large (2e5)
epsilons = [0.05, ]*len(robustness_functions)


Then, the model can be run. Results are saved.

In [ ]:
# Run MORO
with MultiprocessingEvaluator(dike_model) as evaluator:
    results, convergence = evaluator.robust_optimize(robustness_functions,
                                                     scenarios=scenarios,
                                                     nfe=nfe,
                                                     epsilons=epsilons,
                                                     convergence=convergence_metrics,
                                                     population_size=5,
                                                     )

with open(r'generated_datasets/initial_Pareto_policies_test.pkl', 'wb') as file_pi:
    pickle.dump(results, file_pi)

[MainProcess/INFO] pool started with 12 workers
  0%|                                                   | 0/25 [00:00<?, ?it/s]

Now, we can continue with the visualization.

In [ ]:
plt.plot(convergence.epsilon_progress)
plt.xlabel("nr. of generations")
plt.ylabel(r"$\epsilon$ progress")
sns.despine()
plt.show()
sns.set(rc={'figure.figsize':(12,6)})

In [ ]:
df = pd.read_pickle(r'generated_datasets/initial_Pareto_policies_test.pkl')

data = df.loc[:, [o.name for o in robustness_functions]]
limits = parcoords.get_limits(data)
paraxes = parcoords.ParallelAxes(limits)
paraxes.plot(data)
sns.despine()
sns.set(rc={'figure.figsize':(12,6)})

# Scenarios discovery of pareto policies
In this analysis, the pareto set of policies is tested on a set of 10.000 scenarios, covering the full uncertainty space.

First, the results of the policy discovery is loaded.

In [ ]:
pareto_policies = pd.read_pickle(r'generated_datasets/initial_Pareto_policies_test.pkl')

Then, the policies from the analysis are loaded into the model.

In [ ]:
policies = []
for row_number in range(pareto_policies.shape[0]):
    policies.append(
        Policy(name=row_number, **pareto_policies.iloc[row_number, :-5].to_dict())
    )

General settings of the model are set.

In [ ]:
n_scenarios = 50 # maak 10.000

Then, run the model with the robust policies as input, and generate results over a lot of scenarios

In [ ]:
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=n_scenarios, policies=policies)

with open(r'generated_datasets/policy_test_all_scenarios.pkl', 'wb') as file_pi:
    pickle.dump(results, file_pi)

The results are then loaded in for analysis

In [ ]:
scenario_results = pd.read_pickle(r'generated_datasets/policy_test_all_scenarios.pkl')
experiments, outcomes = results

Make a dataframe column for every timestep of the results.

In [ ]:
policies_all_scenarios = pd.DataFrame()
for key, value in outcomes.items():
    temp_df = pd.DataFrame(value, columns=[key + ' '+ str(x) for x in range(len(value[0]))])
    policies_all_scenarios = pd.concat([policies_all_scenarios, temp_df], axis=1)

Add experiments input to the outputs for analysis

In [ ]:
experiments_and_results = pd.concat([experiments,policies_all_scenarios], axis=1)

Combine the variables of different dikes and times

In [ ]:
def combine_columns(dataframe, name):
    combined_columns = []
    for x in dataframe.columns:
        if name in x:
            combined_columns.append(x)

    dataframe['Total ' + str(name)] = dataframe[combined_columns].sum(axis=1)

In [ ]:
combine_columns(policies_all_scenarios, 'Expected Annual Damage')
combine_columns(policies_all_scenarios, 'Dike Investment Costs')
combine_columns(policies_all_scenarios, 'Expected Number of Deaths')
combine_columns(policies_all_scenarios, 'RfR Total Costs')
combine_columns(policies_all_scenarios, 'Expected Evacuation Costs')

Generate statistics

In [ ]:
aggregated_outcomes = pd.concat([experiments[['scenario','policy']],policies_all_scenarios.iloc[:,-5:]], axis=1)

stats_df = pd.DataFrame()
for column in aggregated_outcomes.iloc[:,2:].columns:
    stats_df[column + ' mean'] = aggregated_outcomes.groupby('policy').agg({column:['mean']})
    stats_df[column + ' std'] = aggregated_outcomes.groupby('policy').agg({column:['std']})

A plot with the means and standard deviations of the policies is made, to check reliabilities of the results

In [ ]:
import seaborn.objects as so
(
    so.Plot(aggregated_outcomes, y='Total Expected Annual Damage', x="policy")
    .add(so.Dot(),so.Agg())
    .add(so.Range(),so.Est(errorbar="sd"))
    .layout(size=(12, 6))
)

Boxplots and barplots are made to compare the results of the policies.

In [ ]:
sns.boxplot(aggregated_outcomes, y='Total Expected Annual Damage', x="policy")
sns.set(rc={'figure.figsize':(12,6)})

In [ ]:
sns.boxplot(aggregated_outcomes, y='Total Expected Number of Deaths', x="policy")
sns.set(rc={'figure.figsize': (12, 6)})

In [ ]:
sns.boxplot(aggregated_outcomes, y='Total Expected Evacuation Costs', x="policy")
sns.set(rc={'figure.figsize': (12, 6)})

In [ ]:
sns.catplot(data=stats_df, x='policy', y='Total Dike Investment Costs mean')